In [1]:
# 필요 라이브러리 설치
!pip install konlpy pandas seaborn gensim wordcloud python-mecab-ko wget

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.9/580.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 15.0 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=cbcb7911cc06bbb0ddabdba970ef6702bfa10357b0b6e55f315176f952bb0cab
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [2]:
!pip install konlpy
!pip install mecab-python3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.8/588.8 kB 10.5 MB/s eta 0:00:00


In [3]:
from mecab import MeCab
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from wordcloud import WordCloud
from collections import Counter
import wget, os

In [4]:
os.environ['JAVA_HOME'] = "C:\Program Files\Java\jdk-19"

In [5]:
!sudo apt-get install -y fonts-nanum

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (5,524 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 124950 files and dire

In [6]:
FONT_PATH = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=FONT_PATH, size=10).get_name()
fm.fontManager.addfont(FONT_PATH)
print(font_name)

plt.rcParams['font.family']=font_name
# plt.rcParams['axes.unicode_minus'] == False
assert plt.rcParams['font.family'] == [font_name], "한글 폰트가 설정되지 않았습니다."

NanumGothic


In [7]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 전처리

In [8]:
data = pd.read_csv('/content/drive/MyDrive/Type_of_Inquiry_Classification/train_re.csv')
data.shape

(3706, 6)

In [9]:
label_dict = {
    '코드1': 0,
    '코드2': 0,
    '웹': 1,
    '이론': 2,
    '시스템 운영': 3,
    '원격': 4
}

In [10]:
label_dict.keys()

dict_keys(['코드1', '코드2', '웹', '이론', '시스템 운영', '원격'])

In [11]:
data['new_label'] = data['label'].apply(lambda x: '코드' if x.startswith('코드') else x)
# pd.get_dummies(data['new_label'], drop_first = False, dtype = int)
data = pd.get_dummies(data, columns = ['new_label'], dtype = int)

In [12]:
data.head()

,text,label,length,nouns,pos,morphs,new_label_시스템 운영,new_label_원격,new_label_웹,new_label_이론,new_label_코드
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",코드2,144,"['여기', '커널', '사이즈', '은', '단어', '최대', '길이', '가정...","[('self', 'SL'), ('.', 'SY'), ('convs', 'SL'),...","['self', '.', 'convs', '1', '=', 'nn', '.', 'M...",0,0,0,0,1
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,웹,119,"['이미지', '개', '업로드', '자바', '스크립트', '동적', '폼', '...","[('현재', 'MAG'), ('이미지', 'NNG'), ('를', 'JKO'), ...","['현재', '이미지', '를', '여러', '개', '업로드', '하', '기',...",0,0,1,0,0
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,코드2,147,"['사용', '때', '질문', '포함', '작동', '것', '질문', '제공',...","[('glob', 'SL'), ('.', 'SY'), ('glob', 'SL'), ...","['glob', '.', 'glob', '(', 'PATH', ')', '를', '...",0,0,0,0,1
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",코드2,279,"['오류', '해결']","[('tmpp', 'SL'), ('=', 'SY'), ('tmp', 'SL'), (...","['tmpp', '=', 'tmp', '.', 'groupby', '(', 'by'...",0,0,0,0,1
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,코드2,144,"['안', '폴더', '안', '이미지', '저장', '걸로', '이해', '실행'...","[('filename', 'SL'), ('=', 'SY'), ('TEST', 'SL...","['filename', '=', 'TEST', '_', 'IMAGE', '+', '...",0,0,0,0,1


In [13]:
data = data.drop('label', axis = 1)

In [14]:
data.columns = ['text', 'length',	'nouns', 'pos', 'morphs',	'시스템 운영', '원격',	'웹', '이론', '코드']
data.columns

Index(['text', 'length', 'nouns', 'pos', 'morphs', '시스템 운영', '원격', '웹', '이론',
       '코드'],
      dtype='object')

In [15]:
data.head()

,text,length,nouns,pos,morphs,시스템 운영,원격,웹,이론,코드
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",144,"['여기', '커널', '사이즈', '은', '단어', '최대', '길이', '가정...","[('self', 'SL'), ('.', 'SY'), ('convs', 'SL'),...","['self', '.', 'convs', '1', '=', 'nn', '.', 'M...",0,0,0,0,1
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,119,"['이미지', '개', '업로드', '자바', '스크립트', '동적', '폼', '...","[('현재', 'MAG'), ('이미지', 'NNG'), ('를', 'JKO'), ...","['현재', '이미지', '를', '여러', '개', '업로드', '하', '기',...",0,0,1,0,0
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,147,"['사용', '때', '질문', '포함', '작동', '것', '질문', '제공',...","[('glob', 'SL'), ('.', 'SY'), ('glob', 'SL'), ...","['glob', '.', 'glob', '(', 'PATH', ')', '를', '...",0,0,0,0,1
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",279,"['오류', '해결']","[('tmpp', 'SL'), ('=', 'SY'), ('tmp', 'SL'), (...","['tmpp', '=', 'tmp', '.', 'groupby', '(', 'by'...",0,0,0,0,1
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,144,"['안', '폴더', '안', '이미지', '저장', '걸로', '이해', '실행'...","[('filename', 'SL'), ('=', 'SY'), ('TEST', 'SL...","['filename', '=', 'TEST', '_', 'IMAGE', '+', '...",0,0,0,0,1


- Train Validation split

In [16]:
target = ['시스템 운영', '원격', '웹', '이론', '코드']

x = data['text']
y = data[target]

In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 2023)

In [18]:
x_train.head()

,text
1107,안녕하세요. :)\n금일 수업 자료 중 [1.2 데이터 불러오기 및 분석(EDA)_...
947,'Kernel Panic - not syncing: Attempted to kill...
1225,답변을 받고 여러번 시도해 보았는데 모든 서브넷 설정이 사진과 같은 상태입니다.\n...
1754,"요소가 하나인 튜플을 생성할 때, \nex) score = 90, \n이렇게 한다고..."
704,Q2. 코딩을 작성할 때 첨부 사진처럼 while문을 사용해서 네모칸을 만든 후 주...


- 특수문자 제거

In [19]:
# train 데이터
x_train = x_train.str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
x_train = x_train.str.replace(" +", " ") # 다중 공백 제거

In [20]:
x_train

,text
1107,안녕하세요. :)\n금일 수업 자료 중 [1.2 데이터 불러오기 및 분석(EDA)_...
947,'Kernel Panic - not syncing: Attempted to kill...
1225,답변을 받고 여러번 시도해 보았는데 모든 서브넷 설정이 사진과 같은 상태입니다.\n...
1754,"요소가 하나인 튜플을 생성할 때, \nex) score = 90, \n이렇게 한다고..."
704,Q2. 코딩을 작성할 때 첨부 사진처럼 while문을 사용해서 네모칸을 만든 후 주...
...,...
1953,참고할 교안사항이 있을까요 ㅠ?
2743,아래 사진처럼 python manage.py makemigrations 하는데 아래...
2502,"2)그리고, 보통은 웹서버를 퍼블릭 ip에 만들지만 이 경우에만 프라이빗 ip에 만..."
1561,이런 그래프 라벨 보다 촘촘하게 볼 수 있는 방법은 없을까요...?


- N-grams(sklearn)

In [21]:
def mecab_tokenizer(text):
    pos = mecab.pos(text)
    return [word_pos[0] for word_pos in pos if word_pos[1] in ("NNG", 'NNP','SL')]

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from mecab import MeCab

mecab = MeCab()
count_mecab_vect = CountVectorizer(tokenizer= mecab_tokenizer, ngram_range=(1,2),
                                   stop_words  = ['금일', '안녕', '일차', '간과', '가요', '감사', '구합', '차', 'eda',
                                                  'ㄱ', 'ㄴ', 'ㄷ', 'ㅅ', 'ㄷ', 'ㅁ',
                                                  'ㄷ', 'ㅂ', '가' , '를', '후', '시', '거', '의',  '그', '때',
                                            '로', '요', '과', '걸', '중', '안', '은', '제', '왜', '결', '수', '컴', '터', '제하', '고생'])
x_train_counts = count_mecab_vect.fit_transform(x_train).toarray()
x_test_counts = count_mecab_vect.transform(x_test).toarray()

print(x_train_counts)
print(count_mecab_vect.vocabulary_)
print(x_train_counts.shape)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'수업': 32534, '자료': 38018, '데이터': 25490, '분석': 30113, '유동': 36382, '인구': 37455, '실습': 33623, '문제': 27985, '요일': 35887, '심야': 34057, '이동': 36649, '관련': 23238, '문': 27655, '해당': 45445, '답': 25131, '코드': 42646, '아래': 34112, '작성': 38241, 'seoul': 16162, 'moving': 11616, 'night': 12193, '도착': 25860, '시간': 33190, 'lt': 10606, 'value': 19548, 'counts': 3325, '질문': 40919, '정의': 39398, '기준': 24209, 'groupby': 7045, 'by': 2056, 'as': 1115, 'index': 8537, 'false': 5842, '이동인': 36682, '구': 23472, '합': 45307, 'sum': 17518, '수업 자료': 32569, '자료 데이터': 38035, '데이터 분석': 25623, '분석 유동': 30179, '유동 인구': 36383, '인구 데이터': 37461, '데이터 실습': 25657, '실습 문제': 33723, '문제 요일': 28147, '요일 심야': 35903, '심야 이동': 34058, '이동 관련': 36655, '관련 문': 23264, '문 해당': 27831, '해당 답': 45476, '답 코드': 25165, '코드 아래': 42807, '아래 작성': 34168, '작성 seoul': 38264, 'seoul moving': 16169, 'moving night': 11626, 'night seou

- Machine Learning Modeling

In [23]:
import numpy as np

In [24]:
y_train.values.argmax(axis = 1)

array([4, 1, 2, ..., 2, 4, 2])

- RandomForest Model

In [25]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train_counts, y_train)

RandomForestClassifier()

In [26]:
y_pred = model.predict(x_test_counts)

In [27]:
from sklearn.metrics import f1_score

print(f1_score(y_test, y_pred, average = 'micro'))

0.7209486166007905


- Catboost Model

In [28]:
!pip install CatBoost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.0 MB/s eta 0:00:00


In [29]:
import catboost as cb
from catboost import CatBoostClassifier

cat = cb.CatBoostClassifier(depth = 8)
cat.fit(x_train_counts, y_train.values.argmax(axis = 1))

Learning rate set to 0.083635
0:	learn: 1.5583929	total: 1.43s	remaining: 23m 51s
1:	learn: 1.5102146	total: 2.85s	remaining: 23m 45s
2:	learn: 1.4572561	total: 4.23s	remaining: 23m 26s
3:	learn: 1.4278237	total: 5.73s	remaining: 23m 46s
4:	learn: 1.3884310	total: 8.13s	remaining: 26m 56s
5:	learn: 1.3593736	total: 10.6s	remaining: 29m 22s
6:	learn: 1.3338741	total: 12s	remaining: 28m 22s
7:	learn: 1.3146313	total: 13.4s	remaining: 27m 41s
8:	learn: 1.2980984	total: 14.8s	remaining: 27m 10s
9:	learn: 1.2774216	total: 16.2s	remaining: 26m 40s
10:	learn: 1.2540144	total: 17.5s	remaining: 26m 16s
11:	learn: 1.2377967	total: 18.9s	remaining: 25m 58s
12:	learn: 1.2196356	total: 20.3s	remaining: 25m 40s
13:	learn: 1.2019005	total: 22.5s	remaining: 26m 23s
14:	learn: 1.1874726	total: 25s	remaining: 27m 19s
15:	learn: 1.1753797	total: 26.6s	remaining: 27m 15s
16:	learn: 1.1621107	total: 28s	remaining: 26m 57s
17:	learn: 1.1532520	total: 29.4s	remaining: 26m 42s
18:	learn: 1.1428748	total: 30.8

In [30]:
from sklearn.metrics import f1_score

pred = cat.predict(x_test_counts)

print(f1_score(y_test.values.argmax(axis = 1), pred, average = 'micro'))

0.7897574123989218


- MultinomialNB Model

In [46]:
y_train_counts = y_train.values.argmax(axis = 1)

In [50]:
y_test_counts = y_test.values.argmax(axis = 1)

In [51]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

clf = MultinomialNB()

clf.fit(x_train_counts, y_train_counts)

y_pred = clf.predict(x_test_counts)

print(np.mean(f1_score(y_test_counts, y_pred, average = None)))

0.8047591517795993
